In [1]:
# NOTE: these are not optimized (a new connection is created every time) but should work fine for initial testing

In [2]:
import concurrent.futures
import threading
import psycopg2

In [3]:
# insert data for site
def insert_site_data(domain, robots_content, crawl_delay, sitemap_content):
    conn = psycopg2.connect(host="localhost", user="user", password="SecretPassword")
    conn.autocommit = True

    print("\nInserting values into site")
    cur = conn.cursor()
    cur.execute("INSERT into crawldb.site (domain, robots_content, crawl_delay, sitemap_content) \
                 VALUES (%s, %s, %s, %s)", (domain, robots_content, crawl_delay, sitemap_content))
    
    cur.close()
    conn.close()    

# testing
# insert_site_data("http://bbbb.si", "robots content string", 5, "sitemap content string")
# insert_site_data("http://ccccc.si", "robots content string", 2, "sitemap content string")
insert_site_data("http://ee.si", "robots content string", 5.5, "sitemap content string")


Inserting values into site


In [3]:
# returns crawl delay (float) for specified domain url
def get_crawl_delay(domain_url):
    conn = psycopg2.connect(host="localhost", user="user", password="SecretPassword")
    conn.autocommit = True

    print("Getting crawl delay")
    cur = conn.cursor()

    # get url of first page with code set to FRONTIER
    cur.execute("SELECT crawl_delay FROM crawldb.site WHERE domain = %s", (domain_url,))
    page_url = cur.fetchone()[0]
    
    cur.close()
    conn.close()

    return page_url

print(get_crawl_delay("http://ddd.si"))

Getting crawl delay
5.5


In [4]:
# returns robots.txt content
def get_robots_content(domain_url):
    conn = psycopg2.connect(host="localhost", user="user", password="SecretPassword")
    conn.autocommit = True

    print("Getting robots.txt content")
    cur = conn.cursor()

    # get url of first page with code set to FRONTIER
    cur.execute("SELECT robots_content FROM crawldb.site WHERE domain = %s", (domain_url,))
    robots_content = cur.fetchone()[0]
    
    cur.close()
    conn.close()

    return robots_content

print(get_robots_content("http://bbbb.si"))

Getting robots.txt content
robots content string


In [3]:
# insert url into frontier - inserts into table 'page' with the page_type_code value set to FRONTIER
def insert_page_into_frontier(domain, url):
    conn = psycopg2.connect(host="localhost", user="user", password="SecretPassword")
    conn.autocommit = True

    print("Getting site ID")
    cur = conn.cursor()

    # get id of domain url
    cur.execute("SELECT id FROM crawldb.site WHERE domain= %s", (domain,)) 
    
    # TODO check if domain is stored and return err?
    site_id_result = cur.fetchone()
    if(site_id_result is None):
        print("No domain stored with this url")
        cur.close()
        conn.close()
        return

    site_id = site_id_result[0]
    
    print("Site ID: {}".format(site_id))

    # check if there is already an existing url in the database or frontier
    cur.execute("SELECT id FROM crawldb.page WHERE url = %s", (url,))
    
    id_of_original = cur.fetchone()

    if(id_of_original is None):
         print("Inserting into frontier: {}".format(site_id))
         cur.execute("INSERT into crawldb.page (site_id, url, page_type_code) VALUES (%s, %s, 'FRONTIER')", (site_id, url))
    else:
         print("Exsisting url already in database/frontier")
         # TODO what to do here ?


    cur.close()
    conn.close()


# testing
# insert_page_into_frontier("http://bbbb.si", "https://bbbb.si/page2")
# insert_page_into_frontier("http://bbbb.si", "https://bbbb.si/page3")
# insert_page_into_frontier("http://ccccc.si", "http://ccccc.si/page1")
# insert_page_into_frontier("http://ccccc.si", "http://ccccc.si/page2")
insert_page_into_frontier("http://ee.si", "http://ee.si/binary1")
insert_page_into_frontier("http://ee.si", "http://ee.si/binary2")
insert_page_into_frontier("http://ee.si", "http://ee.si/binary3")
insert_page_into_frontier("http://ee.si", "http://ee.si/binary4")

Getting site ID
Site ID: 5
Inserting into frontier: 5
Getting site ID
Site ID: 5
Inserting into frontier: 5
Getting site ID
Site ID: 5
Inserting into frontier: 5
Getting site ID
Site ID: 5
Inserting into frontier: 5


In [13]:
# when page is crawled, update table 'page' with obtained data
# the function checks for duplicates
def update_page_data(url, page_type_code, html_content, content_hash, http_status_code, accessed_time):
    conn = psycopg2.connect(host="localhost", user="user", password="SecretPassword")
    conn.autocommit = True

    print("Checking duplicate")
    cur = conn.cursor()

    # check if there is a duplicate of content_hash
    cur.execute("SELECT id FROM crawldb.page WHERE content_hash = %s AND html_content IS NOT NULL", \
                 (content_hash,)) # NOTE will we store the hash of duplicates or is there no need?
    original_site = cur.fetchone()
    print(original_site)

    # if duplicate exists
    if original_site is not None:
        page_type_code = 'DUPLICATE'
        original_site_id = original_site[0]
        print("\nInserting page as DUPLICATE")

        # duplicates should have empty html_content column
        cur.execute("UPDATE crawldb.page SET page_type_code= %s, content_hash = %s, http_status_code= %s, accessed_time= %s \
                     WHERE url= %s RETURNING id", ('DUPLICATE', content_hash, http_status_code, accessed_time, url))
        
        # NOTE - this can cause an error if the page hasn't been inserted yet
        id_of_updated_row = cur.fetchone()[0]
        
        # create link from duplicate pointing to 'original' page
        # column names: from_page -> id of duplicate, to_page -> id of original
        cur.execute("INSERT INTO crawldb.link (from_page, to_page) VALUES (%s, %s)", (id_of_updated_row, original_site_id))


    # if there are no duplicates
    else:
        print("\nInserting values into page as {}".format(page_type_code))
        cur.execute("UPDATE crawldb.page SET page_type_code= %s, html_content= %s, content_hash = %s, http_status_code= %s, accessed_time= %s \
                WHERE url= %s", (page_type_code, html_content, content_hash, http_status_code, accessed_time, url))
    



    cur.close()
    conn.close()
    
# testing
# update_page_data("https://bbbb.si/asdasdas/asdas", "HTML", "<head>asdasdas</head>", "aphufogfdfakjnlskxcj", 200, '2022-07-01 00:00:00')
# update_page_data("https://bbbb.si/page2", "HTML", "<head>page 2 bbbb</head>", "oaisdjoaisjdojasodj", 200, '2022-07-01 00:00:00')
update_page_data("http://bbbb.si/page2duplicate3", "HTML", "<head>page4 cccc</head>", "aaa", 200, '2022-07-01 00:00:00.123456')


Checking duplicate
(14,)

Inserting page as DUPLICATE


In [6]:
# if page is binary - update the page_data or image_data tables
def insert_page_data(url, http_status_code, accessed_time, data_type_code):
    conn = psycopg2.connect(host="localhost", user="user", password="SecretPassword")
    conn.autocommit = True

    cur = conn.cursor()

    print("\nInserting values into page as BINARY")
    cur.execute("UPDATE crawldb.page SET page_type_code='BINARY', http_status_code= %s, accessed_time= %s \
                WHERE url= %s RETURNING id", (http_status_code, accessed_time, url))
    
    # NOTE - this can cause an error if the page hasn't been inserted yet
    id_of_updated_row = cur.fetchone()[0]

    cur.execute("INSERT INTO crawldb.page_data (page_id, data_type_code) VALUES (%s, %s)", (id_of_updated_row, data_type_code))
    
    cur.close()
    conn.close()
    
# testing
# update_page_data("https://bbbb.si/asdasdas/asdas", "HTML", "<head>asdasdas</head>", "aphufogfdfakjnlskxcj", 200, '2022-07-01 00:00:00')
# update_page_data("https://bbbb.si/page2", "HTML", "<head>page 2 bbbb</head>", "oaisdjoaisjdojasodj", 200, '2022-07-01 00:00:00')
insert_page_data("http://ee.si/binary2", 200, '2022-07-01 00:00:00.123456', 'DOC')



Inserting values into page as BINARY


In [ ]:
# insert page data v2
def insert_page_data(url, http_status_code, accessed_time, data_type_code):
    cur = None
    try:
        cur = conn.cursor()

        # print("\nInserting values into page as BINARY")
        cur.execute("UPDATE crawldb.page SET page_type_code='BINARY', http_status_code= %s, accessed_time= %s \
                    WHERE url= %s RETURNING id", (http_status_code, accessed_time, url))
        
        # NOTE - this can cause an error if the page hasn't been inserted yet
        id_of_updated_row = cur.fetchone()[0]

        cur.execute("INSERT INTO crawldb.page_data (page_id, data_type_code) VALUES (%s, %s)", (id_of_updated_row, data_type_code))
    except Exception as e:
        print("Error while inserting page data (binary page): ", e)
    finally:
        if cur is not None:
            cur.close()


In [13]:
# if page is an image - update the image_data table
def insert_image_data(url, filename, content_type, accessed_time):
    cur = None
    try:
        cur = conn.cursor()

        # print("\nInserting values into 'page' as BINARY")
        cur.execute("SELECT id FROM crawldb.page WHERE url = %s", (url,))
        
        # NOTE - this can cause an error if the page hasn't been inserted yet
        page_id = cur.fetchone()[0]

        cur.execute("INSERT INTO crawldb.image (page_id, filename, content_type, accessed_time) VALUES (%s, %s, %s, %s)", (page_id, filename, content_type, accessed_time))
        
        cur.close()
    except Exception as e:
        print("Error while inserting image data: ", e)
    finally:
        if cur is not None:
            cur.close()

    
# testing
# update_page_data("https://bbbb.si/asdasdas/asdas", "HTML", "<head>asdasdas</head>", "aphufogfdfakjnlskxcj", 200, '2022-07-01 00:00:00')
# update_page_data("https://bbbb.si/page2", "HTML", "<head>page 2 bbbb</head>", "oaisdjoaisjdojasodj", 200, '2022-07-01 00:00:00')
insert_image_data("http://ee.si/binary4", 200, '2022-07-01 00:00:00.123456', 'image2.jpeg','img/jpg')



Inserting values into 'page' as BINARY


In [ ]:
# get url of first page from frontier
def get_first_page_from_frontier():
    conn = psycopg2.connect(host="localhost", user="user", password="SecretPassword")
    conn.autocommit = True

    print("Getting page url")
    cur = conn.cursor()

    # get url of first page with code set to FRONTIER
    cur.execute("SELECT url FROM crawldb.page WHERE page_type_code = 'FRONTIER'")
    page_url = cur.fetchone()[0]
    
    print(page_url)
    
    cur.close()
    conn.close()

get_first_page_from_frontier()

Getting page url
https://bbbb.si/page3
